In [ ]:
# will use the best 

In [5]:
import os, argparse
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone
from pathlib import Path
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import wandb

import datasets
# from utils import select_device, natural_keys, gazeto3d, angular, getArch
from utils import select_device, natural_keys, gazeto3d, angular, getArch
from model import L2CS

In [6]:

args = argparse.Namespace()
args.gazeMpiimage_dir = '/project/data/Image'  #real data 
args.gazeMpiilabel_dir = '/project/data/Label'  #real label
# args.output = '/project/results/soutput1/snapshots/'
args.dataset = 'mpiigaze'
args.snapshot='/project/results/soutput1/snapshots/'
# args.evalpath = '/project/results/sroutput1/evaluation/'
args.gpu_id = '0,1,2,3'
args.gpu_id = '0'
args.batch_size = 20
args.arch = 'ResNet50'
args.bins=35
args.angle = 180
args.bin_width = 4


In [8]:
batch_size=args.batch_size
arch=args.arch
data_set=args.dataset
# evalpath =args.evalpath
snapshot_path = args.snapshot
bins=args.bins
angle=args.angle
bin_width=args.bin_width

In [9]:
# args = parse_args()
cudnn.enabled = True
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

model_used=getArch(arch, bins)  #resnet50 and 28 bins

In [11]:
#read the fold epoch table
dfile = Path(snapshot_path).parent/"syn_syn_all_MAE.npy"
all_MAE = np.load(dfile)


In [5]:
# check if we have the correct number of checkpoint files 
ppath = snapshot_path
for fold in range(15):
    foldstr = f"fold{fold:0>2}"
    cpath =os.path.join(ppath, foldstr)
    files = os.listdir(cpath)
    print(f'{fold}:{len(files)}',end=" ")

0:60 1:60 2:60 3:60 4:60 5:60 6:60 7:60 8:60 9:60 10:60 11:60 12:60 13:60 14:60 

In [6]:
def get_now():
    now = datetime.utcnow()
    now = now.astimezone(timezone('US/Pacific'))
    date_format='%m/%d/%Y %H:%M:%S'
    now = now.strftime(date_format) 
    return now
print(get_now())

06/27/2022 12:38:23


In [7]:
%%time
all_MAE = []
# tosave={}
for fold in range(15):
    frun = wandb.init(project="52_syn_mode_syn_data")
    frun.name = f'fold{fold}_mae'
    
    print(f"fold={fold}")
    
    epoch_values=[]
    mae_values=[]

    now = get_now()
    
    print(args.gazeMpiilabel_dir)
    folder = os.listdir(args.gazeMpiilabel_dir)
    folder.sort()  #individual label files
    testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder] 
    
    gaze_dataset=datasets.Mpiigaze(testlabelpathcombined, args.gazeMpiimage_dir, transformations, False, angle, fold)

    test_loader = torch.utils.data.DataLoader(
        dataset=gaze_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True)

#     fold_path = os.path.join(evalpath, 'fold' + f'{fold:0>2}'+'/')  #for evaluation
# #     print(f"fold_path is {fold_path}")
#     if not os.path.exists(fold_path):
#         os.makedirs(fold_path)

#     if not os.path.exists(os.path.join(evalpath, f"fold"+str(fold))):
#         os.makedirs(os.path.join(evalpath, f"fold"+str(fold)))

    # list all epoch for testing
    folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
    folder.sort(key=natural_keys)
#     folder.pop(-1)  #remove the tensorboard file, now all snapshot files
#     print(f"folder={folder}")
                    

    softmax = nn.Softmax(dim=1)
#     with open(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".log")), 'w') as outfile:
        
    configuration = (f"\ntest configuration equal gpu_id={gpu}, batch_size={batch_size}, model_arch={arch}\n"
                     f"Start testing dataset={data_set}, FOLD={fold} --{now}---------")
    print(configuration)
#     tosave['time':now]
#     outfile.write(configuration)
    epoch_list=[]
    avg_MAE=[]
    for epoch in folder: 
        print(f"entering epoch={epoch}")
        model=model_used
        checkpoint = torch.load(os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epoch))
        saved_state_dict = checkpoint['model_state_dict']
        model= nn.DataParallel(model,device_ids=[0])
        model.load_state_dict(saved_state_dict)
        model.cuda(gpu)
        model.eval()
        total = 0
        idx_tensor = [idx for idx in range(35)]
        idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
        avg_error = .0
        with torch.no_grad():
            for j, (images, labels, cont_labels, name) in enumerate(test_loader):
                images = Variable(images).cuda(gpu)
                total += cont_labels.size(0)

                label_pitch = cont_labels[:,0].float()*np.pi/180
                label_yaw = cont_labels[:,1].float()*np.pi/180

                gaze_pitch, gaze_yaw = model(images)

                # Binned predictions
                _, pitch_bpred = torch.max(gaze_pitch.data, 1)
                _, yaw_bpred = torch.max(gaze_yaw.data, 1)

                # Continuous predictions
                pitch_predicted = softmax(gaze_pitch)
                yaw_predicted = softmax(gaze_yaw)

                # mapping from binned (0 to 28) to angels (-42 to 42)                
                pitch_predicted = \
                    torch.sum(pitch_predicted * idx_tensor, 1).cpu() * 3 - 52
                yaw_predicted = \
                    torch.sum(yaw_predicted * idx_tensor, 1).cpu() * 3 - 52

                pitch_predicted = pitch_predicted*np.pi/180
                yaw_predicted = yaw_predicted*np.pi/180

                for p,y,pl,yl in zip(pitch_predicted, yaw_predicted, label_pitch, label_yaw):
#                     pl, yl = yl, pl*(-1.0)
#                     yl = yl*(-1.0)
                    avg_error += angular(gazeto3d([p,y]), gazeto3d([pl,yl]))

        x = ''.join(filter(lambda i: i.isdigit(), epoch))
#             print(f"x={x}")
        
        epoch_list.append(x)
        mean_mae = avg_error/total  #mean mae over the 3000 iamges
        avg_MAE.append(mean_mae)  
#             print(f"total={total}")
        now = get_now()
        loger = f"[{epoch}---{args.dataset}] Total Num:{total},MAE:{mean_mae}  {now}"
#         outfile.write(loger)
        print(loger)
#             print(f"done epoch={epochs}")
        epochn = int(x)
#         wandb.log({'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold })
#         tosave ={'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold }
        epoch_values.append(epochn)
        mae_values.append(mean_mae)
# #         wandb.log(tosave)
        wandb.log({'epoch': epoch, f'try3-fold_{fold}_mean_mae':mean_mae}, step=epochn)
       
    all_MAE.append(avg_MAE)
    wandb.finish()


wandb: Currently logged in as: synthesis-ai (use `wandb login --relogin` to force relogin)


fold=0
/project/data/sdata1/Label
34 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=0 --06/27/2022 12:38:27---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2965,MAE:4.160915964395066  06/27/2022 12:38:45
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2965,MAE:5.33510551449734  06/27/2022 12:38:56
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2965,MAE:4.246983338873192  06/27/2022 12:39:08
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2965,MAE:4.236115192593811  06/27/2022 12:39:17
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2965,MAE:3.223275941795794  06/27/2022 12:39:27
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2965,MAE:4.100781316145508  06/27/2022 12:39:38
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2965,MAE:4.755649508429863  06/27/2022 12:39

try3-fold_0_mean_mae,▅█▅▂▆▄▃▂▄▄▂▄▂▂▄▁▂▁▃▁▃▂▁▂▁▂▂▁▂▃▂▁▁▂▃▂▂▃▂▁
epoch,epoch_60.pkl
try3-fold_0_mean_mae,2.9209


fold=1
/project/data/sdata1/Label
29 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=1 --06/27/2022 12:49:56---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2970,MAE:4.214985442861321  06/27/2022 12:50:08
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2970,MAE:3.5431420969877454  06/27/2022 12:50:18
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2970,MAE:3.153305267352677  06/27/2022 12:50:28
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2970,MAE:3.057250446086016  06/27/2022 12:50:38
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2970,MAE:3.1736970597216074  06/27/2022 12:50:48
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2970,MAE:2.9861852528065915  06/27/2022 12:50:59
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2970,MAE:3.0149085547382244  06/27/2022 

try3-fold_1_mean_mae,█▅▃▃▂▂▂▂▂▃▃▂▂▂▁▁▂▃▁▂▁▂▂▂▂▂▂▁▁▂▁▂▁▁▂▂▁▂▂▁
epoch,epoch_60.pkl
try3-fold_1_mean_mae,2.73374


...

fold=13
/project/data/sdata1/Label
36 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=13 --06/27/2022 14:57:45---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2963,MAE:5.238642698029521  06/27/2022 14:57:58
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2963,MAE:3.3351880417867714  06/27/2022 14:58:11
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2963,MAE:3.544214449016411  06/27/2022 14:58:22
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2963,MAE:2.926210910409632  06/27/2022 14:58:35
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2963,MAE:2.893756407494572  06/27/2022 14:58:49
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2963,MAE:2.9984522293649487  06/27/2022 14:59:01
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2963,MAE:3.2209221049859216  06/27/2022

try3-fold_13_mean_mae,█▃▂▂▃▃▂▂▂▁▁▂▁▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,epoch_60.pkl
try3-fold_13_mean_mae,2.60176


fold=14
/project/data/sdata1/Label
31 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=14 --06/27/2022 15:10:41---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:2968,MAE:4.766217458223812  06/27/2022 15:10:54
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:2968,MAE:4.7719846206787215  06/27/2022 15:11:07
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:2968,MAE:3.416069430115444  06/27/2022 15:11:19
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:2968,MAE:3.6964162508978418  06/27/2022 15:11:32
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:2968,MAE:4.4210412299329045  06/27/2022 15:11:45
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:2968,MAE:3.947910450984582  06/27/2022 15:11:58
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:2968,MAE:3.416745725306349  06/27/2022

try3-fold_14_mean_mae,██▃▆▂▇▁▄▃▆▃▄▄▂▂▂▆▅▃▃▃▅▃▃▄▂▂▃▄▃▂▂▃▃▃▃▃▄▃▄
epoch,epoch_60.pkl
try3-fold_14_mean_mae,3.77553


CPU times: user 2h 47min 28s, sys: 22min 39s, total: 3h 10min 7s
Wall time: 2h 45min 13s


In [8]:
all_MAE=np.array(all_MAE)

In [9]:

print(all_MAE.shape)
print(all_MAE.mean(axis=0))
print(all_MAE.mean(axis=1))
print(all_MAE.mean(axis=0).argmin()+1 ,all_MAE.mean(axis=0).min())

(15, 60)
[4.81880487 4.69118491 4.11383053 4.03649961 3.89616644 3.87613096
 3.94570945 3.81186192 3.85713687 3.71216374 3.76881481 3.64245926
 3.65383478 3.79200469 3.69814821 3.62933359 3.70801662 3.69448166
 3.53346677 3.55119761 3.65790676 3.58120133 3.50860357 3.52823219
 3.58579835 3.57960182 3.53693097 3.53672028 3.43278221 3.51364319
 3.49786601 3.5206922  3.52319256 3.51351972 3.45114714 3.40153434
 3.46121521 3.48872513 3.47354852 3.42887746 3.44622823 3.51999128
 3.49112465 3.45206495 3.5177644  3.40166839 3.41582431 3.47636724
 3.40821762 3.41279156 3.41774048 3.49303854 3.4473493  3.43831504
 3.39192038 3.49801706 3.38509793 3.40481096 3.43350378 3.44644004]
[3.41232638 2.95489953 3.48352727 3.96510368 3.60943806 2.90813912
 3.69751067 3.40068128 5.06698743 5.06012454 3.35478976 2.83759362
 3.85699563 2.80234767 3.72735097]
57 3.3850979336223492


In [10]:
best_epoch = all_MAE.mean(axis=0).argmin()+1
min_error = all_MAE.mean(axis=0).min()
mean_errors = all_MAE.mean(axis=0)
print(best_epoch,min_error, mean_errors.shape )

57 3.3850979336223492 (60,)


In [11]:
xx = mean_errors
min_error = 100
with wandb.init(project="52_syn_mode_syn_data") as run:
    run.name = "mean_error_per_epoch"
    for epoch in range(len(xx)):
        err = xx[epoch]
        if err < min_error:
            min_error = err
            run.summary["minimum_error (degree)"] = min_error
        run.log({f'mean epoch error (degree)':err}, step=epoch)
        

mean epoch error (degree),█▇▄▃▄▃▃▃▂▃▂▃▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
mean epoch error (degree),3.44644
minimum_error (degree),3.3851


In [14]:
save_file = Path(snapshot_path).parent/"syn_syn_all_MAE.npy"
with open(save_file, 'wb') as f:
    np.save(f, all_MAE)

In [16]:
for idx, fdata in enumerate(all_MAE):
    print(idx, np.argmin(fdata)+1, fdata.min())

0 34 2.8175722511793686
1 60 2.733735111049329
2 36 3.000589953114087
3 59 3.617358918954061
4 57 3.2385842677050265
5 36 2.4332355778628934
6 49 3.297719393598437
7 41 3.0766720905331026
8 29 4.814923114864992
9 32 4.782696547997513
10 30 2.9867269395008074
11 39 2.4511471773270643
12 8 3.1771322107325326
13 51 2.5392931078252388
14 10 3.189168878736072
